

- 데이터: IMDb (Hugging Face datasets)

- 모델: Embedding → BiLSTM → Linear


In [ ]:
# Colab 사전 설치 & 버전 확인 (2025 기준)
# - PyTorch 2.2+ / Transformers 4.45+ / Datasets 3.0+ 권장
# - 이미 설치되어 있어도 최신으로 맞춤

!pip -q install -U "torch>=2.2, <3.0" "torchvision>=0.17, <1.0" "torchaudio>=2.2, <3.0"
!pip -q install -U "datasets>=3.0.1" "transformers>=4.45.2" "accelerate>=1.0.1" "evaluate>=0.4.2" "scikit-learn>=1.5.2"

In [ ]:
import torch, transformers, datasets, sklearn, evaluate, sys, platform
print("Python            :", sys.version.split()[0])
print("Platform          :", platform.platform())
print("PyTorch           :", torch.__version__)
print("Transformers      :", transformers.__version__)
print("Datasets          :", datasets.__version__)
print("scikit-learn      :", sklearn.__version__)
print("evaluate          :", evaluate.__version__)
print("CUDA available    :", torch.cuda.is_available())
print("GPU device        :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [ ]:
# 라이브러리 임포트 & 시드 고정
import re, math, random, numpy as np
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset

# 재현성을 위해 시드 고정
SEED = 2025
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

In [ ]:
# 데이터 로드: IMDb (Hugging Face datasets)
# load_dataset은 자동으로 캐시 관리해줍니다.
raw = load_dataset("imdb")  # {'train':..., 'test':...}
print(raw)

In [ ]:
# [간단 토크나이저 정의(정규식): 영문 단어 분할
# 영어 소문자/숫자 단어만 추출, 나머지는 공백 처리
token_pattern = re.compile(r"[a-z0-9']+")
def simple_tokenize(text: str):
    # (한국어 주석) 소문자 변환 후 정규식 매칭
    return token_pattern.findall(text.lower())

In [ ]:
# 어휘사전(Vocab) 구축
# 상위 N개 토큰만 사용하여 희소 단어는 <unk> 처리
from collections import Counter
MAX_VOCAB = 30000
PAD, UNK = "<pad>", "<unk>"

# 스페셜 토큰(special token)
# PAD : 짧은 문장이 있다면, 길이를 맞추어주기 위해 패딩 토큰을 사용
# UNK : unknown word (내가 사용하고 있는 사전에 없는 단어)

counter = Counter()
for ex in raw["train"]:
    counter.update(simple_tokenize(ex["text"]))

In [ ]:
itos_ex = ['<pad>', '<unk>','i','love','you']
itos_ex[0]

In [ ]:
# 특수 토큰 포함하여 vocab 생성
most_common = counter.most_common(MAX_VOCAB - 2)
print(most_common)

In [ ]:
print([t for t, _ in most_common])

In [ ]:
print([PAD, UNK] + [t for t, _ in most_common])

In [ ]:
# why "-2" : PAD, UNK 자리 남겨 놓기 위해서
itos = [PAD, UNK] + [t for t, _ in most_common]  # index-to-string (t: token)
print(itos)

In [ ]:
print({t:i for i, t in enumerate(itos)})

In [ ]:
stoi = {t:i for i, t in enumerate(itos)}         # string-to-index
print(stoi)

In [ ]:
PAD_IDX, UNK_IDX = stoi[PAD], stoi[UNK]
print("어휘 크기:", len(itos))

In [ ]:
# 텍스트 -> 인덱스 변환 & 패딩/트렁케이션
MAX_LEN = 256  # 시퀀스 최대 길이
# 모든 리뷰(댓글) 256개 단어 길이로 맞춰줘

def encode(text: str):
    tokens = simple_tokenize(text)
    ids = [stoi.get(tok, UNK_IDX) for tok in tokens][:MAX_LEN]
    # 단어가 있으면 >> 인덱스 가져오고
    # 단어가 없으면 >> UNK_IDX(1) 반환
    if len(ids) < MAX_LEN:
        ids += [PAD_IDX] * (MAX_LEN - len(ids))
        # PAD_IDX : 0
        # MAX_LEN = 256 >> 즉 256 칸 중에서 len(ids)가 4라면 >> 4칸 채우고 252칸 0 채움
    return ids

def encode_label(y: int):
    # IMDb: 0=neg, 1=pos
    return int(y)  # 1 : 긍정, 0 : 부정

In [ ]:
# PyTorch Dataset 래핑
class IMDBTensor(torch.utils.data.Dataset):
    def __init__(self, hf_split):
        self.data = hf_split
    # len() 길이(데이터 개수)
    def __len__(self): return len(self.data)

    # getitem(): 특정 인덱스 데이터 반환
    def __getitem__(self, idx):
        text = self.data[idx]["text"] #영화 리뷰 텍스트
        label = self.data[idx]["label"] # 0 또는 1
        x = torch.tensor(encode(text), dtype=torch.long) # 텍스트를 숫자(텐서의 정수)
        y = torch.tensor(encode_label(label), dtype=torch.long)
        return x, y

# 데이터 셋
train_ds = IMDBTensor(raw["train"])
test_ds  = IMDBTensor(raw["test"])

# 데이터셋을 데이터 로더로 저장
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True,  num_workers=2, pin_memory=torch.cuda.is_available())
test_loader  = DataLoader(test_ds,  batch_size=128, shuffle=False, num_workers=2, pin_memory=torch.cuda.is_available())

In [ ]:
# 모델 정의: 임베딩 + 양방향 LSTM + FC
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, emb=128, hidden=128, num_layers=1, num_classes=2, pad_idx=0, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb, padding_idx=pad_idx)
        # I love you
        # 입력: [45, 250, 120]
        # 임베딩 출력(voca_size * embedded_dimension(fixed=128))
        # [[0.3, 0.2, 1.2, ..... 0.9]
        #  [0.3, 0.2, 1.2, ..... 0.9]
        #  [0.3, 0.2, 1.3, .. .0,0.0]]  # 각 128차원(고정)

        self.lstm = nn.LSTM(emb, hidden, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=0.0)
        # batch_first=True >> batch_size 를 첫 번째 차원으로
        # bidirectional (양방향)?
        # S1 = "이 영화 진짜 재미있다" This movie is great
        # 순방향: 이 >> 영화 >> 진짜 >> 재미있다. (This >> movie >> *is* >> great) 앞에서 뒤로
        # 역방향 : 재미있다 <<  진짜 << 영화 << 이 great << *is* << movie << this  뒤에서 앞으로
        # >> 'not good' : not을 보려면 뒤도 봐야 함 : 맥락을 더 잘 이해함
        # This movie ___ ___  good. (인간) / This movie [mask][mask] good. (컴퓨터)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden*2, num_classes)  # 양방향 → 2배
        # 입력 : 256 (128 forward + 128 backward)
        # num_classes : 2 (부정/긍정)
        # 가중치 초기화(Kaiming 초기화)
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None: nn.init.zeros_(m.bias)
                # bias 는 0으로 초기화

    def forward(self, x):
        # x: (B, T) (배치, 시퀀스 길이) (64, 256)
        e = self.emb(x)                 # (B, T, E) (64, 256, 128)
        out, (h, c) = self.lstm(e)      # h: (num_layers*2, B, H) (2,64,128)
        # out : 각각의 시점(time step) 의 출력값 (사용 안함)
        # (h,c) : h(최종 hidden state **) c: cell state(내부기억, 사용안함)
        # 마지막 레이어의 forward/backward hidden state 결합
        # h = [ [순방향 layer], [역방향 layer]] >> h[0] =순방향 layer h[1]= 역방향 layer
        last_f = h[-2]                  # (B, H) (64, 128) 순방향 마지막
        last_b = h[-1]                  # (B, H) (64, 128) 역방향 마지막
        h_cat = torch.cat([last_f, last_b], dim=1)  # (B, 2H) # (64, 256)
        h_cat = self.dropout(h_cat)
        logits = self.fc(h_cat)         # (B, 2) (64,2)
        # 예) [[0.1. 1.5]]
        return logits

model = BiLSTM(len(itos), emb=128, hidden=128, num_layers=1, pad_idx=PAD_IDX).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
# 학습/평가 루프
def train_one_epoch(epoch):
    model.train()
    total_loss = total_correct = total = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * y.size(0)
        # y.size(0) : batch_size
        total_correct += (logits.argmax(1) == y).sum().item()
        # logits.argmax(1)
        # logits = [[0.5,1.5],[2.0,-0.5]]
        # argmax = [1,0] # 각 행에서 가장 큰 값의 인덱스
        total += y.size(0)
    print(f"[Train] Epoch {epoch} | loss={total_loss/total:.4f} | acc={total_correct/total:.4f}")

@torch.no_grad()
def evaluate():
    model.eval()
    total_loss = total_correct = total = 0
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = criterion(logits, y)
        total_loss += loss.item() * y.size(0)
        total_correct += (logits.argmax(1) == y).sum().item()
        total += y.size(0)
    print(f"[Test ] loss={total_loss/total:.4f} | acc={total_correct/total:.4f}")

EPOCHS = 3  # 2~3 에폭 권장
for ep in range(1, EPOCHS+1):
    train_one_epoch(ep)
    evaluate()


In [ ]:
# EOS